# Demo 1 (from no-frameworks to frameworks)

#### Pre-requisites:

Run [this colab](https://colab.research.google.com/github/openai/openai-cookbook/blob/main/examples/vector_databases/cassandra_astradb/Philosophical_Quotes_cassIO.ipynb) on your database at least up to populating the (non-partitioned) table (i.e. the "Insert quotes into vector store" cell).

In [1]:
import os
import cassio

cassio.init(
    token=os.environ['ASTRA_DB_APPLICATION_TOKEN'],
    database_id=os.environ['ASTRA_DB_ID'],
    keyspace=os.environ.get('ASTRA_DB_KEYSPACE'),
)

In [2]:
# create a vector store with cassIO
from cassio.table import MetadataVectorCassandraTable

v_table = MetadataVectorCassandraTable(table="philosophers_cassio", vector_dimension=1536)

In [3]:
import openai

openai.api_key = os.environ['OPENAI_API_KEY']

embedding_model_name = "text-embedding-ada-002"
completion_model_name = "gpt-3.5-turbo"

## Quote generator, no frameworks

In [4]:
def find_quote_and_author(query_quote, n, author=None, tags=None):
    query_vector = openai.Embedding.create(                                            # <=== Direct OpenAI (embeddings)
        input=[query_quote],
        engine=embedding_model_name,
    ).data[0].embedding
    metadata = {}
    if author:
        metadata["author"] = author
    if tags:
        for tag in tags:
            metadata[tag] = True
    #
    results = v_table.ann_search(
        query_vector,
        n=n,
        metadata=metadata,
    )
    return [
        (result["body_blob"], result["metadata"]["author"])
        for result in results
    ]

In [5]:
generation_prompt_template = """"Generate a single short philosophical quote on the given topic,
similar in spirit and form to the provided actual example quotes.
Do not exceed 20-30 words in your quote.

REFERENCE TOPIC: "{topic}"

ACTUAL EXAMPLES:
{examples}
"""

In [6]:
def generate_quote(topic, n=2, author=None, tags=None):
    quotes = find_quote_and_author(query_quote=topic, n=n, author=author, tags=tags)
    if quotes:
        prompt = generation_prompt_template.format(
            topic=topic,
            examples="\n".join(f"  - {quote[0]}" for quote in quotes),
        )
        # a little logging:
        print("** quotes found:")
        for q, a in quotes:
            print(f"**    - {q} ({a})")
        print("** end of logging")
        #
        response = openai.ChatCompletion.create(                                       # <=== Direct OpenAI (LLM)
            model=completion_model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=320,
        )
        return response.choices[0].message.content.replace('"', '').strip()
    else:
        print("** no quotes found.")
        return None

### Testing

In [7]:
q_topic = generate_quote("animals", author="schopenhauer")
print("\nA new generated quote:")
print(q_topic)

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun! (schopenhauer)
**    - The assumption that animals are without rights, and the illusion that our treatment of them has no moral significance, is a positively outrageous example of Western crudity and barbarity. Universal compassion is the only guarantee of morality. (schopenhauer)
** end of logging

A new generated quote:
In the eyes of morality, animals are not mere objects to be exploited; their intrinsic essence dem

## LangChain-ify

_Note: this example is designed so that the table population in the pre-requisites has a schema compatible with what'll come!_

In [8]:
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Cassandra
from langchain.prompts import PromptTemplate

Create the LangChain "vector store"

In [9]:
oai_embeddings = OpenAIEmbeddings(model=embedding_model_name)
oai_llm = OpenAI(model_name=completion_model_name)

vec_store = Cassandra(
    table_name='philosophers_cassio',
    embedding=oai_embeddings,
    session=None,  # = get defaults from init()
    keyspace=None,  # = get defaults from init()
)

/home/stefano/.virtualenvs/genai-learning-day-3-10/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/stefano/.virtualenvs/genai-learning-day-3-10/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [10]:
prompt_template = """Generate a single short philosophical quote on the given topic,
similar in spirit and form to the provided actual example quotes.
Do not exceed 20-30 words in your quote.

REFERENCE TOPIC: "{question}"

ACTUAL EXAMPLES:
{context}

ANSWER:
"""

philo_prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": philo_prompt}

### The core of the app, LangChain version

In [11]:
def generate_quote(topic, n=2, author=None, tags=None):
    # Nontrivial metadata construction
    # Awkward for customization (nesting of dictionaries, and the partitioned case is worse)
    # must fiddle with prompt to handle the no-results case (hallucination risk)
    if tags:
        _from_tags = {t: True for t in tags}
    else:
        _from_tags = {}
    if author:
        _from_author = {'author': author}
    else:
        _from_author = {}
    #
    vstore_search_kwargs = {
        **{'k': n},
        **_from_author,
        **_from_tags,
    }
    qa = RetrievalQA.from_chain_type(
        llm=oai_llm,
        chain_type="stuff",
        retriever=vec_store.as_retriever(search_kwargs=vstore_search_kwargs),          #  <== not the most intuitive thing to do, eh?
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    answer = qa({'query': topic})
    if answer['source_documents']:
        # a little logging:
        print("** quotes found:")
        for q in answer['source_documents']:
            print(f"**    - {q.page_content})")
        print("** end of logging")
        #
        return answer['result'].strip()
    else:
        print("** no quotes found.")
        return None

### Testing

In [12]:
generate_quote('animals')

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun!)
**    - Animals are in possession of themselves; their soul is in possession of their body. But they have no right to their life, because they do not will it.)
** end of logging


'Animals possess wisdom beyond our comprehension, yet we deny their inherent rights, suppressing the divine essence that dwells within their being.'

In [13]:
generate_quote('animals', author='spinoza')

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun!)
**    - Animals are in possession of themselves; their soul is in possession of their body. But they have no right to their life, because they do not will it.)
** end of logging


'"The true measure of humanity lies in our treatment of animals, recognizing their inherent worth and granting them the dignity they deserve."'

In [14]:
generate_quote('animals', author='spinoza', tags=['politics'])

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun!)
**    - Animals are in possession of themselves; their soul is in possession of their body. But they have no right to their life, because they do not will it.)
** end of logging


'In the eyes of philosophy, animals are reduced to mere objects, stripped of their inherent worth and used for our own desires. Let us acknowledge their essence and value, shining through every living being.'

In [15]:
generate_quote('animals', tags=['politics'])

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun!)
**    - Animals are in possession of themselves; their soul is in possession of their body. But they have no right to their life, because they do not will it.)
** end of logging


'Animals are not just possessions, but beings with souls, deserving of respect and recognition of their inherent worth.'